In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

PATH = "D:/study/2020 Spring/597 Deep Learning/dataset/cats_and_dogs_filtered"

In [2]:
##setting the dir of the datasets; 
train_dir = "D:/study/2020 Spring/597 Deep Learning/dataset/cats_and_dogs_filtered/train"
validation_dir = "D:/study/2020 Spring/597 Deep Learning/dataset/cats_and_dogs_filtered/validation"

train_cats_dir = "D:/study/2020 Spring/597 Deep Learning/dataset/cats_and_dogs_filtered/train/cats"  # directory with our training cat pictures
train_dogs_dir = "D:/study/2020 Spring/597 Deep Learning/dataset/cats_and_dogs_filtered/train/dogs"  # directory with our training dog pictures
validation_cats_dir = "D:/study/2020 Spring/597 Deep Learning/dataset/cats_and_dogs_filtered/validation/cats"  # directory with our validation cat pictures
validation_dogs_dir = "D:/study/2020 Spring/597 Deep Learning/dataset/cats_and_dogs_filtered/validation/dogs"  # directory with our validation dog pictures

In [3]:
#take a peek at the data; 
print(len(os.listdir(train_cats_dir)))
#load images using ImageDataGenerator; 
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
batch_size = 1000
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')
validation_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

1000
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [4]:
import tensorflow.keras as keras
from keras import models
from keras import layers
sample_training_images, sample_training_labels = next(train_data_gen)
sample_validation_images, sample_validation_labels = next(validation_data_gen)
sample_training_images = sample_training_images.reshape((1000, 150 * 150 *3))
sample_training_images = sample_training_images.astype('float32') / 255
sample_validation_images = sample_validation_images.reshape((1000, 150 * 150 *3))
sample_validation_images = sample_validation_images.astype('float32') / 255


Using TensorFlow backend.


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

clf = LogisticRegression()
clf.fit(sample_training_images,sample_training_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
test_predict = clf.predict(sample_validation_images)
print("Accuracy:",metrics.accuracy_score(sample_validation_labels, test_predict))

Accuracy: 0.59


In [7]:
from sklearn import svm

clf = svm.SVC(kernel='linear')
clf.fit(sample_training_images,sample_training_labels)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [8]:
test_predict = clf.predict(sample_validation_images)
print("Accuracy:",metrics.accuracy_score(sample_validation_labels, test_predict))

Accuracy: 0.569


In [9]:
network3 = models.Sequential()
network3.add(layers.Dense(1, activation='linear', kernel_regularizer=keras.regularizers.l2()))

sample_training_labels_2 = np.where(sample_training_labels==1, 1, -1)
# Hinge loss
def hinge_loss(y_true, y_pred):    
    return tf.maximum(0., 1- y_true*y_pred)

# Train the model
network3.compile(optimizer='adam', loss=hinge_loss)
network3.fit(sample_training_images, sample_training_labels_2, epochs=5, verbose=False)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










In [10]:
y_pred = network3.predict(sample_validation_images)
y_pred = np.where(y_pred>0,1,0)

In [11]:
print("Accuracy:",metrics.accuracy_score(sample_validation_labels, y_pred))

Accuracy: 0.493


In [12]:
from keras.utils import to_categorical
sample_training_labels = to_categorical(sample_training_labels)
sample_validation_labels = to_categorical(sample_validation_labels)

In [13]:
network = models.Sequential() # Sequential feed forward NN
network.add(layers.Dense(2, activation='sigmoid', input_shape=(150*150*3,)))
network.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
network.fit(sample_training_images, sample_training_labels, epochs=5, batch_size=128)


Epoch 1/5
1000/1000 [==============================] - 1s 535us/step - loss: 0.6979 - acc: 0.4820
Epoch 2/5
1000/1000 [==============================] - 0s 409us/step - loss: 0.6921 - acc: 0.5190
Epoch 3/5
1000/1000 [==============================] - 0s 390us/step - loss: 0.6902 - acc: 0.5230
Epoch 4/5
1000/1000 [==============================] - 0s 394us/step - loss: 0.6878 - acc: 0.5370
Epoch 5/5
1000/1000 [==============================] - 0s 400us/step - loss: 0.6864 - acc: 0.5660


In [14]:
test_loss, test_acc = network.evaluate(sample_validation_images, sample_validation_labels)
print(test_acc)

1000/1000 [==============================] - 0s 310us/step
0.582


In [15]:
network2 = models.Sequential() # Sequential feed forward NN
network2.add(layers.Dense(2, activation='linear', input_shape=(150*150*3,)))
network2.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
network2.fit(sample_training_images, sample_training_labels, epochs=5, batch_size=128)

Epoch 1/5
1000/1000 [==============================] - 1s 522us/step - loss: 7.3492 - acc: 0.5010
Epoch 2/5
1000/1000 [==============================] - 0s 406us/step - loss: 8.1558 - acc: 0.4960
Epoch 3/5
1000/1000 [==============================] - 0s 401us/step - loss: 8.1558 - acc: 0.4960
Epoch 4/5
1000/1000 [==============================] - 0s 400us/step - loss: 8.1558 - acc: 0.4960
Epoch 5/5
1000/1000 [==============================] - 0s 402us/step - loss: 8.1558 - acc: 0.4960


In [16]:
test_loss, test_acc = network2.evaluate(sample_validation_images, sample_validation_labels)
print(test_acc)

1000/1000 [==============================] - 0s 324us/step
0.507
